In [57]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
df = pd.read_csv('data/declarations.csv')
common_offices = pd.read_csv('data/common_offices.csv', converters={"person_id": eval})

In [4]:
office_names = df[['office_id', 'office_name']].drop_duplicates().reset_index(drop=True)
office_names.head()

,office_id,office_name
0,14,Государственная Дума
1,449,Президент Российской Федерации
2,118,Государственное собрание – Курултай Республик...
3,267,Законодательное собрание Алтайского края
4,211,Губернатор Иркутской области


In [5]:
person_names = df[['person_id', 'person_name']].drop_duplicates().reset_index(drop=True)
person_names.head()

,person_id,person_name
0,8,Зюганов Геннадий Андреевич
1,9,Алферов Жорес Иванович
2,10,Андреев Андрей Анатольевич
3,11,Апарина Алевтина Викторовна
4,12,Афонин Юрий Вячеславович


In [7]:
common_offices.year.value_counts()

2017    1730
2014    1630
2015    1507
2016    1454
2013     648
2012     558
2011     398
2010     300
2009     226
2018      81
2008      30
2006       2
1999       1
1998       1
Name: year, dtype: int64

In [22]:
YEAR = 2010

year_offices = common_offices[common_offices.year == YEAR]

In [36]:
graph = nx.Graph()

In [37]:
edge_properties = year_offices[['year', 'office_id']]
edge_lists = year_offices.person_id
for year, office_id, nodes_to_connect in year_offices.values:
    combs = list(itertools.permutations(nodes_to_connect,2))
    graph.add_edges_from(combs)

In [39]:
print(graph.order())
print(graph.size())

2732
408477


In [60]:
centrality = nx.eigenvector_centrality(graph)
print(pd.Series(centrality).describe())

count    2.732000e+03
mean     1.022180e-02
std      1.617538e-02
min      1.138235e-30
25%      9.960445e-22
50%      7.169933e-07
75%      3.581223e-02
max      3.585920e-02
dtype: float64


In [61]:
top_centrality_nodes = pd.Series(centrality).nlargest(10)
top_centrality_ids = np.array(top_centrality_nodes.index)
top_centrality_names = person_names[person_names.person_id.isin(top_centrality_ids)]
top_centrality_names

,person_id,person_name
0,8,Зюганов Геннадий Андреевич
1,9,Алферов Жорес Иванович
2,10,Андреев Андрей Анатольевич
3,11,Апарина Алевтина Викторовна
4,12,Афонин Юрий Вячеславович
5,13,Гаврилов Сергей Анатольевич
7,15,Гостев Руслан Георгиевич
9,17,Денисенко Олег Иванович
10,18,Езерский Николай Николаевич
80,89,Жириновский Владимир Вольфович
